In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import time
from datetime import datetime
import random 

# Functions for EDA

In [2]:
def time_df(df):
    df1 = df.copy()
    df1['starttime'] = pd.to_datetime(df1['starttime'], errors="coerce")
    df1['stoptime'] = pd.to_datetime(df1['stoptime'], errors="coerce")
    df1['start hour'] = df1['starttime'].dt.hour
    df1['stop hour'] = df1['stoptime'].dt.hour
    
    df1["start date"] = pd.to_datetime(df1['start date'], errors="coerce")
    df1["start_date"] = df1["start date"].dt.date
    df1['start year'] = df1['start date'].dt.year
    df1['start month'] = df1['start date'].dt.month
    df1['start day'] = df1['start date'].dt.day
    df1['start dayofweek'] = df1['start date'].dt.weekday+1   # Monday:1 ~ Sunday: 7
    df1['Isweekday'] = [0 if 6<=x<=7 else 1 for x in df1['start dayofweek']]
    df1['stop date'] = df1['start_date']
    
    return df1

In [3]:
def aggregate_df(df):
    # Define distance: if same start and end location, use average speed of 8.3mph to estimate distance.
    # If different locations, calculate Manhattan distance between two stations
    df['distance']=np.where(df['start station id'] == df['end station id'],df['tripduration']*8.3/3600, abs(df['start station longitude']-df['end station longitude'])*53+abs(df['start station latitude']-df['end station latitude'])*69)
    
    df_daily = df.groupby('start_date').agg({'tripduration':['count','mean'],'distance':'mean'}).reset_index()
    df_daily.columns = ['start_date','trip per day','daily avg trip duration','daily avg distance']
    df_daily_merged = df.merge(df_daily, how = 'left', on = 'start_date')
    
    df_hourly = df.groupby(['start station id','start hour']).agg({'tripduration':['count','mean'],'distance':'mean'}).reset_index()
    df_hourly.columns = ['start station id','start hour','trip per hour','hourly avg trip duration','hourly avg distance']
    df_hourly_merged = df_daily_merged.merge(df_hourly, how = 'left', on = ['start station id','start hour'])
    df_hourly_eachday = df.groupby(['start station id','start_date','start hour']).\
    agg({'tripduration':['count','mean'],'distance':'mean'}).reset_index()
    df_hourly_eachday.columns = ['start station id','start_date','start hour','trip per hour eachday',
                                 'hourly avg trip duration eachday','hourly avg distance eachday']
    df_hourly_eachday_merged = df_hourly_merged.merge(df_hourly_eachday, how = 'left',
                                                      on = ['start station id','start_date','start hour'])
    return df_hourly_eachday_merged

In [4]:
 def merged_bikecount(df):
    checkout = df.groupby(['start station id','start_date','start hour'])['tripduration'].count().reset_index()
    checkout.columns = ['start station id','start_date','start hour','checkout counts']
    checkin = df.groupby(['end station id','stop date','stop hour'])[['tripduration']].count().reset_index()
    checkin.columns=['end station id','stop date','stop hour','checkin counts']
    start_end = pd.merge(checkout, checkin,  how='outer', left_on=['start station id','start_date','start hour'], 
                    right_on = ['end station id','stop date','stop hour'])
    start_end['start station id'] = start_end['start station id'].fillna(start_end['end station id'])
    start_end['start_date'] = start_end['start_date'].fillna(start_end['stop date'])
    start_end['start hour'] = start_end['start hour'].fillna(start_end['stop hour'])
    start_end['checkout counts'] = start_end['checkout counts'].fillna(0)
    start_end['checkin counts'] = start_end['checkin counts'].fillna(0)
    start_end = start_end.drop(['end station id','stop date','stop hour'],axis=1)
    start_end.columns=['station id','date','hour','checkout counts','checkin counts']
    start_end['total counts'] = start_end['checkout counts'] + start_end['checkin counts']
    merged_start_end = pd.merge(df, start_end,  how='left', left_on=['start station id','start_date','start hour'], 
         right_on = ['station id','date','hour']).drop(['station id','date','hour'],axis = 1)
    merged_start_end = pd.merge(merged_start_end, start_end,  how='left', left_on=['end station id','stop date','stop hour'], 
         right_on = ['station id','date','hour']).drop(['station id','date','hour'],axis = 1) 
    merged_start_end = merged_start_end.rename(columns={'checkout counts_x':'start station checkout counts',
                                                    'checkin counts_x':'start station checkin counts',
                                                    'total counts_x':'start station total counts', 
                                                    'checkout counts_y':'end station checkout counts',
                                                    'checkin counts_y':'end station checkin counts',
                                                    'total counts_y':'end station total counts'}) 
    return merged_start_end

# Merge Monthly Datasets (Trip and Weather)

In [5]:
trip_csv_files_2019 = [('2019' + "%.2d" + '-citibike-tripdata.csv') % i for i in range(9, 13)]
trip_csv_files_2020 = [('2020' + "%.2d" + '-citibike-tripdata.csv') % i for i in range(1, 9)]
csv_files = trip_csv_files_2019 + trip_csv_files_2020

In [6]:
# Read monthly Citi Bike trip data, apply some filters.
# Randomly select 5% of each month trip data and save it as csv for each month. 
# Similar method used to make test data for machine learning validation.

path = 'finaldata/'
for i, csv in enumerate(csv_files):
    df_trip = pd.read_csv(path+csv)
    
    # remove trips that are longer than 1 day 
    df_trip = df_trip.loc[df_trip['tripduration']<= 24*3600] 
    
    # remove areas that are not in NYC
    df_trip = df_trip.loc[(df_trip['start station latitude']>40) & (df_trip['start station latitude']<41)] 
    df_trip = df_trip.loc[(df_trip['end station latitude']>40) & (df_trip['end station latitude']<41)] 
    
    # sampling 5% dataset
    df_sample = df_trip.sample(frac=0.05, random_state = 42)
    
    # add custom features using self-defined functions
    df_sample = time_df(df_sample)
    df_sample = aggregate_df(df_sample)
    df_sample = merged_bikecount(df_sample)
    
    df_sample.to_csv(csv[:6] + '-clean-sample.csv')
    print('Data extraction ' + csv)

Data extraction 201909-citibike-tripdata.csv
Data extraction 201910-citibike-tripdata.csv
Data extraction 201911-citibike-tripdata.csv
Data extraction 201912-citibike-tripdata.csv
Data extraction 202001-citibike-tripdata.csv
Data extraction 202002-citibike-tripdata.csv
Data extraction 202003-citibike-tripdata.csv
Data extraction 202004-citibike-tripdata.csv
Data extraction 202005-citibike-tripdata.csv
Data extraction 202006-citibike-tripdata.csv
Data extraction 202007-citibike-tripdata.csv
Data extraction 202008-citibike-tripdata.csv


In [7]:
#df_sample_tmp = time_df(df_sample)
#df_sample_tmp = aggregate_df(df_sample_tmp)
#df_sample_tmp

In [8]:
#checkout = df_sample_tmp.groupby(['start station id','start date','start hour'])['tripduration'].count().reset_index()
#checkout.columns = ['start station id','start date','start hour','checkout counts']
#checkin = df_sample_tmp.groupby(['end station id','stop date','stop hour'])[['tripduration']].count().reset_index()
#checkin.columns=['end station id','stop date','stop hour','checkin counts']
#print(checkin.dtypes)

In [9]:
#start_end = pd.merge(checkout, checkin,  how='outer', left_on=['start station id','start date','start hour'], 
                     #right_on = ['end station id','stop date','stop hour'])
#start_end

In [10]:
clean_sample_list = []
for i, csv in enumerate(csv_files):
    clean_sample_list.append(csv[:6] + '-clean-sample.csv')

In [11]:
# Merge all months clean_sample.csv into one clean_sample dataframe
clean_sample_df = pd.DataFrame()
for i in range(len(clean_sample_list)):
    if csv.endswith('.csv'):
        temp_df = pd.read_csv(clean_sample_list[i], index_col = 0)
        clean_sample_df = pd.concat([clean_sample_df, temp_df], axis = 0)
        print('Completed ' + str(i) + ' dataframe')
clean_sample_df.to_csv('clean_sample.csv')

Completed 0 dataframe
Completed 1 dataframe
Completed 2 dataframe
Completed 3 dataframe
Completed 4 dataframe
Completed 5 dataframe
Completed 6 dataframe
Completed 7 dataframe
Completed 8 dataframe
Completed 9 dataframe
Completed 10 dataframe
Completed 11 dataframe
